In [ ]:
%load_ext autoreload
%autoreload 2

from src.data_processing import get_data_to_buffer, BufferDataset
from src.configs import TrainConfig

In [3]:
buffer = get_data_to_buffer(TrainConfig())

dataset = BufferDataset(buffer)

  5%|▍         | 604/13100 [00:01<00:34, 359.19it/s]


KeyboardInterrupt: 

In [11]:
from torch.utils.data import DataLoader
from functools import partial
from src.data_processing import collate_fn_tensor

loader = DataLoader(
    dataset,
    batch_size=TrainConfig.batch_expand_size * TrainConfig.batch_size,
    shuffle=True,
    collate_fn=partial(collate_fn_tensor, batch_expand_size=TrainConfig.batch_expand_size),
    drop_last=True,
    num_workers=0
)

In [13]:
batch = next(iter(loader))[0]
batch["text"].shape

torch.Size([16, 174])

In [20]:
from src.FFT import FFTBlock
import torch

hidden_size = 16
intermediate_size = 64
n_head = 4
batch_size = 4
seq_len = 12

fft_block = FFTBlock(hidden_size, intermediate_size, n_head)

inp_tensor = torch.rand(batch_size, seq_len, hidden_size, dtype=torch.float32)

out_tensor = fft_block(inp_tensor)

assert inp_tensor.shape == out_tensor.shape

torch.Size([4, 12, 16])
torch.Size([4, 12, 16])
torch.Size([4, 12, 16])


In [19]:
out_tensor.shape

torch.Size([12, 16])

In [2]:
from src.configs import TrainConfig, FastSpeech2Config, MelSpectrogramConfig
from src.training.training_loop import train, prepare

model_config = FastSpeech2Config()
train_config = TrainConfig()
mel_config = MelSpectrogramConfig()

model, train_loader, fastspeech_loss, optimizer, scheduler = prepare(model_config, train_config, mel_config)

100%|██████████| 13100/13100 [02:07<00:00, 102.61it/s]

cost 127.79s to load all data into buffer.


In [13]:
train_config = TrainConfig()

In [3]:
for el in train_loader:
    break

In [4]:
db = el[0]

character = db["text"].long().to(train_config.device)
mel_target = db["mel_target"].float().to(train_config.device)
duration = db["duration"].int().to(train_config.device)
mel_pos = db["mel_pos"].long().to(train_config.device)
src_pos = db["src_pos"].long().to(train_config.device)
max_mel_len = db["mel_max_len"]

In [5]:
mel_output, duration_predictor_output = model(character,
                                            src_pos,
                                            mel_pos=mel_pos,
                                            mel_max_length=max_mel_len,
                                            length_target=duration)


In [7]:
mel_target.shape

torch.Size([16, 865, 80])

In [12]:
src_pos.shape

torch.Size([16, 184])

In [6]:
mel_output.shape

torch.Size([16, 865, 80])

In [8]:
model = train(train_config, model, train_loader, fastspeech_loss, optimizer, scheduler)

  0%|          | 1/1600000 [00:00<132:57:16,  3.34it/s]

: 

: 